# Spotify Machine Learning

Model: k-nearest-neighbors

The user's top n amount of songs will be training set for the model. The user will then select a genre they are interested in discovering new music in.

In [43]:
# Libraries

%matplotlib inline

import json

import pandas as pd
from pandas.io.json import json_normalize

import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [56]:
# Import JSON Here
with open("sample.json") as f:
    sample_json = json.load(f)
    
sample = json_normalize(sample_json) # converts to pandas format

with open("3_songs.json") as f:
    three_songs_json = json.load(f)
    
three_songs = json_normalize(three_songs_json['audio_features'])

In [55]:
# Interpret JSON Data Here
sample

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.011,https://api.spotify.com/v1/audio-analysis/11dF...,0.696,207960,0.905,11dFghVXANMlKmJXsNCbNl,0.000905,2,0.302,-2.743,1,0.103,114.944,4,https://api.spotify.com/v1/tracks/11dFghVXANMl...,audio_features,spotify:track:11dFghVXANMlKmJXsNCbNl,0.625


In [20]:
three_songs

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.000273,https://api.spotify.com/v1/audio-analysis/7ouM...,0.366,366213,0.963,7ouMYWpwJ422jRcDASZB7P,0.01220,11,0.1150,-5.301,0,0.1420,137.114,4,https://api.spotify.com/v1/tracks/7ouMYWpwJ422...,audio_features,spotify:track:7ouMYWpwJ422jRcDASZB7P,0.211
1,0.000202,https://api.spotify.com/v1/audio-analysis/4VqP...,0.602,304840,0.905,4VqPOruhp5EdPBeR92t6lQ,0.06400,2,0.1170,-4.046,1,0.0775,128.019,4,https://api.spotify.com/v1/tracks/4VqPOruhp5Ed...,audio_features,spotify:track:4VqPOruhp5EdPBeR92t6lQ,0.411
2,0.002420,https://api.spotify.com/v1/audio-analysis/2tak...,0.585,237040,0.842,2takcwOaAZWiXQijPHIx7B,0.00686,9,0.0866,-5.883,0,0.0556,118.211,4,https://api.spotify.com/v1/tracks/2takcwOaAZWi...,audio_features,spotify:track:2takcwOaAZWiXQijPHIx7B,0.428


## Characteristics to Analyze

X_variables:
1. acousticness
2. danceability
3. duration_ms
4. energy
5. instrumentalness
6. liveness
7. loudness
8. speechiness
9. tempo 
10. valence

Predicting: Ranking 

Assume all attributes of are of equal weight initially.

In [24]:
three_songs_train = three_songs[['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']]

In [30]:
three_songs_train

,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,0.000273,0.366,366213,0.963,0.01220,0.1150,-5.301,0.1420,137.114,0.211
1,0.000202,0.602,304840,0.905,0.06400,0.1170,-4.046,0.0775,128.019,0.411
2,0.002420,0.585,237040,0.842,0.00686,0.0866,-5.883,0.0556,118.211,0.428


## K-Nearest Neighbors


In [41]:
# Normalize the Data

normalized_three_songs = (three_songs_train-three_songs_train.mean())/three_songs_train.std()

In [42]:
normalized_three_songs

,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,-0.548959,-1.152290,0.983010,0.985945,-0.490684,0.517540,-0.238952,1.119862,0.987193,-1.151833
1,-0.605283,0.640724,0.033156,0.027540,1.150561,0.635162,1.097830,-0.316144,0.025140,0.505481
2,1.154243,0.511566,-1.016166,-1.013486,-0.659878,-1.152702,-0.858878,-0.803718,-1.012333,0.646352


In [61]:
sample_test = sample[['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']]